In [1]:
import numpy as np
import pandas as pd
import boto3

In [2]:
# Let's use Amazon S3
# The Access Key ID and Secret Access Key are in ~/.aws/credentials
# Region is set in ~/.aws/config
s3 = boto3.resource('s3')

In [3]:
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

athena-query-bucket-testda24
aws-glue-assets-936480984004-eu-north-1
hacksprint-2023
temp-transformation-rs
vehicle-bucket-alarm


In [4]:
# Print the file names in the bucket
revolvos_bucket = s3.Bucket('hacksprint-2023')
for object in revolvos_bucket.objects.all():
    print(object.key)

Telematics Data Dictionary 2023_02_14.xlsx
Vehicle_Fault.csv
Vehicle_Fault_Alarm.csv
Vehicle_Fault_Alarm_Item.csv
Vehicle_Location.csv
Vehicle_Specs.csv


In [5]:
%%time
# Load one of the files into a dataframe
obj = s3.Object('hacksprint-2023', 'Vehicle_Fault_Alarm_Item.csv').get()
df = pd.read_csv(obj['Body'])

KeyboardInterrupt: 

In [6]:
%%time
# Load from the local CSV file instead.
df = pd.read_csv('../S3 original bucket/Vehicle_Fault_Alarm_Item.csv')

CPU times: user 29.1 s, sys: 2.96 s, total: 32 s
Wall time: 32.4 s


In [7]:
print(df.dtypes)

FAULT_ALARM_ID        int64
SNAP_SHOT_ITEM_ID     int64
ITEM_VALUE           object
dtype: object


In [8]:
print(df.head())

   FAULT_ALARM_ID  SNAP_SHOT_ITEM_ID ITEM_VALUE
0      2384228315                  1    531.301
1      2384228315                  2   4687.189
2      2384228315                  3         30
3      2384228315                  4       1234
4      2384228315                  5          4


In [10]:
print(df.shape)

(228523951, 3)


Insert magical cleaning...

In [ ]:
%%time
# Send the cleaned dataframe content back to a new CSV file on the S3 bucket
csv_clean = df.to_csv(index=False)
_ = s3.Object('hacksprint-2023', 'Vehicle_Fault_Alarm_Item-clean.csv').put(Body=csv_clean)